# Solving a Load Balancing Problem with D-Wave Quantum Computer

There are $n$ jobs $\{J_1,\ldots, J_n\}$ and a single resource. Each job $J_i$ has a release date $r_i$ and a deadline $d_i$. The processing time of each job is 1 time unit, and requires 1 unit from the resource throughout its execution.

We have a determine an integer starting time $S_i$ for each job $J_i$ such that $r_i\leq S_i \leq d_i-1$, and if $\ell(t)$ equals the number of jobs starting at time $t$, then $\sum_{t=0}^T \ell(t)^2$ is minimized.

In [1]:
import dimod
import random
from dwave.system import LeapHybridCQMSampler

In [2]:
TOKEN="DEV-ec968c99419e738f54bfe5ef26e15ae8c86b3d2f"

## Build a Constrained Quadratic Model

A mathematical formulation of the Load Balancing Problem is
$$
\begin{split}
& \min \sum_{j=1}^T \left(\sum_{i=1}^n x_{ij}\right)^2\\
& \sum_{j=1}^T x_{ij} =1,\quad \forall i \in [n]\\
& x_{ij} = 0,\quad j \notin [r_i,d_i-1]\\
& x_{ij} \in \{0,1\}
\end{split}
$$



In [3]:
def build_model(jobs : list, T : int):
    cqm = dimod.ConstrainedQuadraticModel()
    n = len(jobs)
    x = [[dimod.Binary(f'x_{i}_{j}') for j in range(T)] for i in range(n) ]
    i = 0
    for r,d in jobs:
        cqm.add_constraint(sum([x[i][j] for j in range(r, d)]) == 1 )
        i=i+1
    cqm.set_objective(sum([sum([x[i][j] for i in range(n) if j >= jobs[i][0] and j<jobs[i][1]]) **2 for j in range(T) ]))

    return cqm

def write_data(jobs: list, file : str):
    with open(file, 'w') as f:
        f.write(str(len(jobs)))
        f.write('\n')
        for r, d in jobs:
            f.write(str(r) + " " + str(d) + '\n')
        f.close()

## Generate jobs at random

In [19]:
n = 200
horizon = 0
jobs = []
for i in range(n):
    r = random.randint(1,30)
    d = r + random.randint(10,20)
    jobs.append([r,d])
    if horizon < d:
        horizon = d
print(jobs, horizon)
write_data(jobs,"data_3.txt")

[[26, 37], [2, 18], [1, 16], [19, 37], [9, 23], [6, 26], [23, 39], [8, 27], [9, 25], [6, 22], [3, 15], [1, 15], [13, 33], [19, 30], [4, 16], [4, 15], [15, 28], [21, 37], [20, 31], [24, 36], [23, 40], [2, 13], [7, 17], [23, 35], [20, 33], [29, 40], [15, 30], [9, 27], [18, 28], [9, 19], [25, 44], [29, 46], [2, 12], [15, 31], [30, 50], [2, 16], [26, 39], [6, 16], [13, 24], [22, 38], [22, 40], [25, 42], [29, 39], [22, 34], [21, 33], [14, 25], [7, 17], [17, 33], [19, 36], [28, 46], [3, 15], [27, 37], [23, 35], [26, 38], [16, 26], [16, 27], [8, 26], [11, 31], [19, 37], [12, 32], [7, 17], [9, 26], [4, 19], [19, 31], [20, 39], [9, 20], [16, 32], [30, 49], [4, 18], [14, 27], [18, 32], [20, 30], [7, 21], [7, 23], [11, 25], [2, 15], [8, 23], [19, 32], [11, 29], [15, 31], [12, 23], [22, 39], [9, 27], [16, 31], [17, 36], [7, 24], [19, 29], [6, 22], [22, 42], [1, 18], [14, 30], [20, 37], [9, 22], [8, 22], [30, 45], [5, 25], [2, 16], [12, 31], [7, 22], [8, 23], [14, 31], [1, 19], [16, 35], [2, 13], [

In [20]:
cqm = build_model(jobs,horizon)

In [6]:
sampler = LeapHybridCQMSampler(token=TOKEN)

In [21]:
import time

start_t = time.time()
sampleset = sampler.sample_cqm(cqm, time_limit=10, label="Load Balancing")
end_t = time.time()
print('Solving using CPU took %d seconds' % (end_t-start_t))

Solving using CPU took 0 seconds


In [22]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)  

if len(feasible_sampleset):      
    best = feasible_sampleset.first
    print("{} feasible solutions of {}.".format(
        len(feasible_sampleset), len(sampleset)))
    print("best solution value: ", best.energy)
    
    load = [0]*horizon
    i = 0
    for r,d in jobs:
        for j in range(r,d):
            if best.sample[f'x_{i}_{j}']==1:
                load[j] += 1
        i=i+1
        
    obj = 0
    for j in range(horizon):
        obj += load[j]**2
    print("obj calc: " , obj)

66 feasible solutions of 103.
best solution value:  878.0
obj calc:  878
